## Import packages

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

Import Tensorflow and the Keras classes needed to construct our model.

In [2]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

from tensorflow.keras import layers
from tensorflow.keras import Model, metrics
from tensorflow.keras.optimizers import RMSprop, SGD

import os
import zipfile

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Load data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
train_local_zip = '/content/drive/MyDrive/lulc-classification.zip'
zip_ref = zipfile.ZipFile(train_local_zip, 'r')
zip_ref.extractall('/content/sample_data/lulc-classification.zip')

test_local_zip = '/content/drive/MyDrive/test.zip'
zip_ref = zipfile.ZipFile(test_local_zip, 'r')
zip_ref.extractall('/content/sample_data/lulc-tra-val/lulc-classificatio')

zip_ref.close()

In [6]:
PATH = '/content/sample_data/lulc-classification.zip/lulc-classification'

After extracting its contents, assign variables with the proper file path for the training and validation set.

In [7]:
train_dir = os.path.join(PATH, 'train')
test_dir = os.path.join(PATH, 'test')

In [8]:
train_BarrenLands_dir = os.path.join(train_dir, '1_BarrenLands___jpeg')

In [9]:
num_barren_tr = len(os.listdir(train_BarrenLands_dir))

print('total training barren images:', num_barren_tr)

total training barren images: 354


In [10]:
batch_size = 32
img_height = 160
img_width = 160

## Training and Validation Generators

Now that you know the images you are dealing with, it is time for you to code the generators that will fed these images to your Network. For this, complete the `train_val_generators` function below:

**Important Note:** The images have a resolution of 300x300 but the `flow_from_directory` method you will use allows you to set a target resolution. In this case, **set a `target_size` of (150, 150)**. This will heavily lower the number of trainable parameters in your final network, yielding much quicker training times without compromising the accuracy!

In [11]:
# GRADED FUNCTION: train_val_generators
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
  ### START CODE HERE

  # Instantiate the ImageDataGenerator class 
  # Don't forget to normalize pixel values and set arguments to augment the images 
  train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

  # Pass in the appropriate arguments to the flow_from_directory method
  train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=batch_size, 
                                                      class_mode='categorical',
                                                      target_size=(img_width, img_height))

  # Instantiate the ImageDataGenerator class (don't forget to set the rescale argument)
  # Remember that validation data should not be augmented
  validation_datagen = ImageDataGenerator(
      rescale=1./255,
      )

  # Pass in the appropriate arguments to the flow_from_directory method
  validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                batch_size=batch_size, 
                                                                class_mode='categorical',
                                                                target_size=(img_width, img_height))
  ### END CODE HERE
  return train_generator, validation_generator


In [12]:
# Test your generators
train_generator, test_generator = train_val_generators(train_dir, test_dir)

Found 10266 images belonging to 29 classes.
Found 0 images belonging to 0 classes.


**Expected Output:**
```
Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.
```

## Transfer learning - Create the pre-trained model

Download the `inception V3` weights into the `/tmp/` directory:

In [15]:
import tensorflow_hub as hub

m = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_m/feature_vector/2",
                   trainable=False),  # Can be True, see below.
      # Add a fully connected layer with 1024 hidden units and ReLU activation
    layers.Dense(512, activation='relu'),
    # Add a dropout rate of 0. 2
    layers.Dropout(0.2),

    layers.Dense(29, activation='softmax')
])
m.build([None, 480, 480, 3])  # Batch input shape.

In [16]:
m.compile(optimizer='Adam',
          loss='categorical_crossentropy',
          metrics = ['accuracy'])

In [18]:
# Run this and see how many epochs it should take before the callback
# fires, and stops training at 99.9% accuracy
# (It should take a few epochs)
initial_epochs = 5
history = m.fit(train_generator,
                    #batch_size = batch_size,
                    #steps_per_epoch = num_barren_tr // batch_size,
                    epochs = initial_epochs,
                    #validation_steps = num_barren_val // batch_size,
                    verbose = 1)

Epoch 1/5
321/321 [==============================] - 145s 353ms/step - loss: 1.3717 - accuracy: 0.5697
Epoch 2/5
321/321 [==============================] - 108s 336ms/step - loss: 0.8936 - accuracy: 0.6998
Epoch 3/5
321/321 [==============================] - 107s 333ms/step - loss: 0.7810 - accuracy: 0.7387
Epoch 4/5
321/321 [==============================] - 107s 334ms/step - loss: 0.7146 - accuracy: 0.7504
Epoch 5/5
321/321 [==============================] - 108s 337ms/step - loss: 0.6691 - accuracy: 0.7650


The training should have stopped after less than 10 epochs and it should have reached an accuracy over 99,9% (firing the callback). This happened so quickly because of the pre-trained model you used, which already contained information to classify humans from horses. Really cool!

Now take a quick look at the training and validation accuracies for each epoch of training:

You will need to submit this notebook for grading. To download it, click on the `File` tab in the upper left corner of the screen then click on `Download` -> `Download .ipynb`. You can name it anything you want as long as it is a valid `.ipynb` (jupyter notebook) file.

**Congratulations on finishing this week's assignment!**

You have successfully implemented a convolutional neural network that leverages a pre-trained network to help you solve the problem of classifying humans from horses.

**Keep it up!**

In [19]:
m.trainable = True

m.compile(optimizer=tf.keras.optimizers.Adam(1e-4),  # Low learning rate
          loss = 'categorical_crossentropy', 
          metrics = ['accuracy']
)

history_fine = m.fit(train_generator,
                    #batch_size = batch_size,
                    initial_epoch = history.epoch[-1],
                    #steps_per_epoch = num_barren_tr // batch_size,
                    #validation_steps = num_barren_val // batch_size,
                    epochs = 20,
                    verbose = 1)

Epoch 5/20
321/321 [==============================] - 354s 982ms/step - loss: 2.1758 - accuracy: 0.5414
Epoch 6/20
321/321 [==============================] - 314s 977ms/step - loss: 0.8753 - accuracy: 0.7117
Epoch 7/20
321/321 [==============================] - 314s 977ms/step - loss: 0.6659 - accuracy: 0.7736
Epoch 8/20
321/321 [==============================] - 314s 977ms/step - loss: 0.5630 - accuracy: 0.8072
Epoch 9/20
321/321 [==============================] - 316s 983ms/step - loss: 0.5147 - accuracy: 0.8201
Epoch 10/20
321/321 [==============================] - 316s 984ms/step - loss: 0.4516 - accuracy: 0.8432
Epoch 11/20
321/321 [==============================] - 316s 983ms/step - loss: 0.4012 - accuracy: 0.8549
Epoch 12/20
321/321 [==============================] - 316s 984ms/step - loss: 0.3946 - accuracy: 0.8625
Epoch 13/20
321/321 [==============================] - 316s 984ms/step - loss: 0.3634 - accuracy: 0.8754
Epoch 14/20
321/321 [==============================] - 316s 

In [13]:
def test_generate(TEST_DIR):
  test_datagen = ImageDataGenerator(
      rescale=1./255)

  # Pass in the appropriate arguments to the flow_from_directory method
  test_generator = test_datagen.flow_from_directory(directory=TEST_DIR,
                                                      class_mode='categorical',
                                                      batch_size = 1,
                                                      target_size=(img_width, img_height),
                                                      shuffle = False,
                                                      classes=['.'])
  return test_generator

In [14]:
test_generator = test_generate(test_dir)

Found 1618 images belonging to 1 classes.


In [20]:
dictionary = {0 : 10, 1 : 11, 2 : 12, 3 : 13, 4 : 14, 5 : 15, 6 : 16, 7 : 17, 8 : 18, 9 : 19, 10 : 1, 11 : 20, 12 : 21, 13 : 22, 14 : 23, 15 : 24, 16 : 25, 17 : 26, 18 : 27, 19 : 28, 20 : 29, 21 : 2, 22 : 3, 23 : 4, 24 : 5, 25 : 6, 26 : 7, 27 : 8, 28 : 9}

In [21]:
predict_proba = m.predict_generator(test_generator)
predict = np.argmax(predict_proba, axis = 1)
predict = np.vectorize(dictionary.get)(predict)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  """Entry point for launching an IPython kernel.


In [22]:
submission = pd.DataFrame()

filenames = list()
for filename in test_generator.filenames:
  if filename.endswith(".jpg"):
    filenames.append(filename[2:])
submission["id.jpg"] = filenames

submission["label"] = predict

submission.to_csv("/content/drive/MyDrive/submission_effnet_m_adam.csv", index = False)

In [23]:
from google.colab import files
files.download("/content/drive/MyDrive/submission_effnet_m_adam.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>